# Editing a T5 QA model with GRACE

In [1]:
import grace
from grace.editors import GRACE_barebones as GRACE
from grace.utils import tokenize_qa
import torch
import copy
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-small-ssm-nq")
tokenizer = AutoTokenizer.from_pretrained("google/t5-small-ssm-nq")

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

C:\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
layer_to_edit = "encoder.block[4].layer[1].DenseReluDense.wo" # Which layer to edit?
init_epsilon = 3.0 # Initial epsilon for GRACE codebook entries
learning_rate = 1.0 # Learning rate with which to learn new GRACE values
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
original_model = copy.deepcopy(model)

In [4]:
# --- wrap model with GRACE ---
edited_model = GRACE(model, layer_to_edit, init_epsilon, learning_rate, device, generation=False)

In [5]:
# --- Desired edit ---
edit_input = {
    "text": ["How tall is the empire state building?"],
    "labels": ["1,454 feet"],
}

edit_tokens = tokenize_qa(edit_input, tokenizer, device)

# --- Check model's prediction for this edit before applying the edit ---
preds = original_model.generate(edit_tokens["input_ids"]).squeeze()
original_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"Before Editing. Question: {edit_input['text'][0]}. Answer: {original_answer}")

Before Editing. Question: How tall is the empire state building?. Answer: 71 ft


C:\Python311\Lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:
# --- Apply the edit ---
edited_model.edit(edit_tokens)

AttributeError: 'GRACEAdaptor' object has no attribute 'weight'

In [ ]:
# --- Check model's prediction for this edit AFTER applying the edit ---
preds = edited_model.generate(edit_tokens["input_ids"]).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"After Editing. Question: {edit_input['text'][0]}. Answer: {new_answer}")

In [8]:
# --- Trying slightly different input text ---
test_input = {
    "text": ["how high is the empire state building?"],
    "labels": ["1,454 feet"]
}

test_tokens = tokenize_qa(test_input, tokenizer, device)

preds = edited_model.generate(test_tokens["input_ids"], max_length=20).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"After Editing. Question: {test_input['text'][0]}. Answer: {new_answer}")

After Editing. Question: how high is the empire state building?. Answer: 57 ft


In [9]:
# --- Check if the original and edited model have the same prediction on an unrelated input ---
unrelated_input = {
    "text": ["How tall is the eiffel tower?"],
    "labels": ["1,083 feet"]
}

unrelated_tokens = tokenize_qa(unrelated_input, tokenizer, device)

preds = original_model.generate(unrelated_tokens["input_ids"]).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"Before Editing. Question: {unrelated_input['text'][0]}. Answer: {new_answer}")

preds = edited_model.generate(unrelated_tokens["input_ids"]).squeeze()
new_answer = tokenizer.decode(preds, skip_special_tokens=True)
print(f"After Editing. Question: {unrelated_input['text'][0]}. Answer: {new_answer}")

Before Editing. Question: How tall is the eiffel tower?. Answer: 157 ft
After Editing. Question: How tall is the eiffel tower?. Answer: 157 ft
